In [1]:
import os
import time
import shutil
import numpy as np
import tensorflow as tf
import core.utils as utils
from tqdm import tqdm
from core.dataset2 import Dataset
from core.yolov3 import YOLOv3, decode, compute_loss
from core.config2 import cfg

In [2]:
trainset = Dataset('train')
validset = Dataset("val")

logdir = "./data/log/try1/"

train_steps_per_epoch = len(trainset) # traindataset step 개수 24186 // Batch_size
valid_steps_per_epoch = len(validset)

global_steps = tf.Variable(1, trainable=False, dtype=tf.int64)
warmup_steps = cfg.TRAIN.WARMUP_EPOCHS * train_steps_per_epoch
total_steps = cfg.TRAIN.EPOCHS * train_steps_per_epoch

input_tensor = tf.keras.layers.Input([416, 416, 3])
conv_tensors = YOLOv3(input_tensor)

output_tensors = []
for i, conv_tensor in enumerate(conv_tensors):
    pred_tensor = decode(conv_tensor, i)
    output_tensors.append(conv_tensor)
    output_tensors.append(pred_tensor)

model = tf.keras.Model(input_tensor, output_tensors)
model.load_weights("./yolov3")
optimizer = tf.keras.optimizers.Adam()
if os.path.exists(logdir): shutil.rmtree(logdir)
writer = tf.summary.create_file_writer(logdir)

def train_step(image_data, target):
    with tf.GradientTape() as tape:
        pred_result = model(image_data, training=True)
        giou_loss=conf_loss=prob_loss=0

        # optimizing process
        for i in range(3):
            conv, pred = pred_result[i*2], pred_result[i*2+1]
            loss_items = compute_loss(pred, conv, *target[i], i)
            giou_loss += loss_items[0]
            conf_loss += loss_items[1]
            prob_loss += loss_items[2]

        total_loss = giou_loss + conf_loss + prob_loss

        gradients = tape.gradient(total_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        if global_steps % 200==1:
            tf.print(
                f"| Step: %.5d\t| \
                Epochs: %.2d /%.2d\t| \
                lr: %.6f\t| \
                Giou_Loss: %.3f\t| \
                Conf_Loss: %.3f\t| \
                Prob_Loss: %.3f\t| \
                Total_Loss: %.3f \t|\
                "%(global_steps,epoch,cfg.TRAIN.EPOCHS,optimizer.lr.numpy(),giou_loss,conf_loss,prob_loss,total_loss)
            )

        
        # update learning rate
        global_steps.assign_add(1)
        if global_steps < warmup_steps:
            lr = global_steps / warmup_steps *cfg.TRAIN.LR_INIT
        else:
            lr = cfg.TRAIN.LR_END + 0.5 * (cfg.TRAIN.LR_INIT - cfg.TRAIN.LR_END) * (
                (1 + tf.cos((global_steps - warmup_steps) / (total_steps - warmup_steps) * np.pi))
            )
        optimizer.lr.assign(lr.numpy())

        # writing summary data
        with writer.as_default():
            tf.summary.scalar("lr", optimizer.lr, step=global_steps)
            tf.summary.scalar("train_loss/total_loss", total_loss, step=global_steps)
            tf.summary.scalar("train_loss/giou_loss", giou_loss, step=global_steps)
            tf.summary.scalar("train_loss/conf_loss", conf_loss, step=global_steps)
            tf.summary.scalar("train_loss/prob_loss", prob_loss, step=global_steps)
        writer.flush()

def val_step(validset):
    giou_loss=conf_loss=prob_loss=0
    
    for image_data, target in validset:
        pred_result = model(image_data, training=False)
        
        # optimizing process
        for i in range(3):
            conv, pred = pred_result[i*2], pred_result[i*2+1]
            loss_items = compute_loss(pred, conv, *target[i], i)
            giou_loss += loss_items[0]
            conf_loss += loss_items[1]
            prob_loss += loss_items[2]

        total_loss = giou_loss + conf_loss + prob_loss
    
    giou_loss = giou_loss / valid_steps_per_epoch
    conf_loss = conf_loss / valid_steps_per_epoch
    prob_loss = prob_loss / valid_steps_per_epoch
    total_loss = total_loss / valid_steps_per_epoch
    
    tf.print(
        f"| Epochs: %.2d /%.2d\t| \
        Giou_Loss: %4.2f\t| \
        Conf_Loss: %4.2f\t| \
        Prob_Loss: %4.2f\t| \
        Total_Loss: %4.2f \t|\
        "%(epoch,cfg.TRAIN.EPOCHS,giou_loss,conf_loss,prob_loss,total_loss)
    )

    # writing summary data
    with writer.as_default():
        tf.summary.scalar("valid_loss/total_loss", total_loss, step=epoch)
        tf.summary.scalar("valid_loss/giou_loss", giou_loss, step=epoch)
        tf.summary.scalar("valid_loss/conf_loss", conf_loss, step=epoch)
        tf.summary.scalar("valid_loss/prob_loss", prob_loss, step=epoch)
    writer.flush()        
        

for epoch in range(cfg.TRAIN.EPOCHS):
    for image_data, target in trainset:
        train_step(image_data, target)
    valid_step(validset)
    model.save_weights("./yolov3")
# http://localhost:6006/

C:\Users\rh987\AppData\Roaming\Python\Python39\site-packages\albumentations\imgaug\transforms.py:346: FutureWarning: This IAAAffine is deprecated. Please use Affine instead
  warnings.warn("This IAAAffine is deprecated. Please use Affine instead", FutureWarning)


| Step: 00001	|                 Epochs: 00 /10	|                 lr: 0.001000	|                 Giou_Loss: 66.562	|                 Conf_Loss: 56.302	|                 Prob_Loss: 37.774	|                 Total_Loss: 160.638 	|                
| Step: 00201	|                 Epochs: 00 /10	|                 lr: 0.000017	|                 Giou_Loss: 32.954	|                 Conf_Loss: 47.737	|                 Prob_Loss: 21.674	|                 Total_Loss: 102.364 	|                
| Step: 00401	|                 Epochs: 00 /10	|                 lr: 0.000033	|                 Giou_Loss: 25.494	|                 Conf_Loss: 31.810	|                 Prob_Loss: 21.136	|                 Total_Loss: 78.440 	|                
error []
| Step: 00601	|                 Epochs: 00 /10	|                 lr: 0.000050	|                 Giou_Loss: 56.350	|                 Conf_Loss: 48.231	|                 Prob_Loss: 35.998	|                 Total_Loss: 140.579 	|                
| Step: 00801	|     

KeyboardInterrupt: 

In [8]:
model.save_weights("./yolov3_try2")

In [3]:
!tensorboard --logdir ./data/log

^C


In [4]:
for epoch in range(1): #cfg.TRAIN.EPOCHS
#     for image_data, target in trainset:
#         train_step(image_data, target)
    val_step(validset)
    # model.save_weights("./yolov3")

KeyboardInterrupt: 

# Test

In [1]:
import cv2
import os
import shutil
import numpy as np
import tensorflow as tf
import core.utils as utils
from core.config import cfg
from core.yolov3 import YOLOv3, decode

In [3]:
INPUT_SIZE   = 416
NUM_CLASS    = len(utils.read_class_names(cfg.YOLO.CLASSES))
CLASSES      = utils.read_class_names(cfg.YOLO.CLASSES)

predicted_dir_path = './predicted'
ground_truth_dir_path = './ground-truth'
if os.path.exists(predicted_dir_path): shutil.rmtree(predicted_dir_path)
if os.path.exists(ground_truth_dir_path): shutil.rmtree(ground_truth_dir_path)
if os.path.exists(cfg.TEST.DECTECTED_IMAGE_PATH): shutil.rmtree(cfg.TEST.DECTECTED_IMAGE_PATH)

os.mkdir(predicted_dir_path)
os.mkdir(ground_truth_dir_path)
os.mkdir(cfg.TEST.DECTECTED_IMAGE_PATH)

# Build Model
input_layer  = tf.keras.layers.Input([INPUT_SIZE, INPUT_SIZE, 3])
feature_maps = YOLOv3(input_layer)

bbox_tensors = []
for i, fm in enumerate(feature_maps):
    bbox_tensor = decode(fm, i)
    bbox_tensors.append(bbox_tensor)

model = tf.keras.Model(input_layer, bbox_tensors)
model.load_weights("./yolov3_try2")

with open(cfg.TEST.ANNOT_PATH, 'r') as annotation_file:
    for num, line in enumerate(annotation_file):
        annotation = line.strip().split()
        image_path = annotation[0]
        image_name = "/".join(image_path.split('/')[-2:])
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        bbox_data_gt = np.array([list(map(int, box.split(','))) for box in annotation[1:]])

        if len(bbox_data_gt) == 0:
            bboxes_gt=[]
            classes_gt=[]
        else:
            bboxes_gt, classes_gt = bbox_data_gt[:, :4], bbox_data_gt[:, 4]
        ground_truth_path = os.path.join(ground_truth_dir_path, str(num) + '.txt')

        print('=> ground truth of %s:' % image_name)
        num_bbox_gt = len(bboxes_gt)
        with open(ground_truth_path, 'w') as f:
            for i in range(num_bbox_gt):
                class_name = CLASSES[classes_gt[i]]
                xmin, ymin, xmax, ymax = list(map(str, bboxes_gt[i]))
                bbox_mess = ' '.join([class_name, xmin, ymin, xmax, ymax]) + '\n'
                f.write(bbox_mess)
                print('\t' + str(bbox_mess).strip())
        print('=> predict result of %s:' % image_name)
        predict_result_path = os.path.join(predicted_dir_path, str(num) + '.txt')
        # Predict Process
        image_size = image.shape[:2]
        image_data = utils.image_preporcess(np.copy(image), [INPUT_SIZE, INPUT_SIZE])
        image_data = image_data[np.newaxis, ...].astype(np.float32)

        pred_bbox = model.predict(image_data)
        pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred_bbox]
        pred_bbox = tf.concat(pred_bbox, axis=0)
        bboxes = utils.postprocess_boxes(pred_bbox, image_size, INPUT_SIZE, cfg.TEST.SCORE_THRESHOLD)
        bboxes = utils.nms(bboxes, cfg.TEST.IOU_THRESHOLD, method='nms')


        if cfg.TEST.DECTECTED_IMAGE_PATH is not None:
            image = utils.draw_bbox(image, bboxes)
            cv2.imwrite(cfg.TEST.DECTECTED_IMAGE_PATH+image_name, image)

        with open(predict_result_path, 'w') as f:
            for bbox in bboxes:
                coor = np.array(bbox[:4], dtype=np.int32)
                score = bbox[4]
                class_ind = int(bbox[5])
                class_name = CLASSES[class_ind]
                score = '%.4f' % score
                xmin, ymin, xmax, ymax = list(map(str, coor))
                bbox_mess = ' '.join([class_name, score, xmin, ymin, xmax, ymax]) + '\n'
                f.write(bbox_mess)
                print('\t' + str(bbox_mess).strip())
        f.close()

=> ground truth of visdrone/VisDrone2019-MOT-test\sequences\uav0000009_03358_v\uav0000009_03358_v_0000001.jpg:
	car 130 672 303 760
	car 178 600 333 665
	car 317 272 418 331
	car 338 231 447 277
	car 363 193 461 241
	car 774 262 883 333
	car 765 216 863 281
	car 754 182 856 244
	car 1048 591 1176 682
	car 1022 510 1156 596
	car 1278 280 1359 348
	car 1243 186 1347 245
	people 344 295 366 332
	car 1202 120 1299 180
	car 1184 97 1277 140
=> predict result of visdrone/VisDrone2019-MOT-test\sequences\uav0000009_03358_v\uav0000009_03358_v_0000001.jpg:
1/1 [==============================] - 2s 2s/step
	people 0.7549 0 0 1359 764
	people 0.4810 0 342 68 764
	people 0.2036 958 0 1029 195
	car 1.0000 467 0 474 764
	car 1.0000 470 0 470 764
	car 1.0000 0 0 1359 764
	car 1.0000 1227 0 1230 764
	car 1.0000 776 0 793 764
	car 1.0000 1302 290 1313 317
	car 1.0000 1307 0 1307 764
	car 1.0000 113 0 304 764
	car 1.0000 225 0 245 764
	car 1.0000 261 0 261 764
	car 1.0000 53 16 155 16
	car 1.0000 0 14 12

KeyboardInterrupt: 

In [3]:
!tensorboard --logdir ./data/log

^C
